In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geojsonio
import config

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [3]:
mapbox_access_token = config.mapbox_access_token
plotly.tools.set_credentials_file(username=config.plotly_username, api_key=config.plotly_api_key)

In [4]:
malaysia = dict(lat=4, lon=110, zoom=4.8)
west_malaysia = dict(lat=4, lon=102, zoom=6)

In [5]:
cn = ["bn", "ph", "gs"]
cn_colors = {
    "bn": "rgba(9, 39, 129, 0.5)", 
    "ph": "rgba(237, 28, 36, 0.5)", 
    "gs": "rgba(0, 136, 0, 0.5)"
} 

In [6]:
def get_map(year, area=malaysia):
    data = go.Data([go.Scattermapbox(lat=area["lat"], lon=area["lon"], mode='markers')])
    layout = go.Layout(
        height=800, width=1100, autosize=False, hovermode="closest",
        mapbox=dict(
            layers=[{
                "source": f"https://raw.githubusercontent.com/theianchan/ge14-map/master/data/ge{year}-geo-{c}.geojson",
                "sourcetype": "geojson",
                "type": "fill",
                "color": cn_colors[c]
            } for c in cn],
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(lat=area["lat"], lon=area["lon"]),
            zoom=area["zoom"],
            style="light",
            pitch=0
        ),
    )
    return dict(data=data, layout=layout)

# GE14

In [6]:
py.iplot(get_map("14"), filename="GE14")

# GE 13

In [7]:
py.iplot(get_map("13"), filename="GE13")

# GE 12

In [8]:
py.iplot(get_map("12"), filename="GE12")

# GE 11

In [9]:
py.iplot(get_map("11"), filename="GE11")

# GE 14 (West Malaysia)

In [10]:
py.iplot(get_map("14", area=west_malaysia), filename="GE14-WM")

# GE 13 (West Malaysia)

In [11]:
py.iplot(get_map("13", area=west_malaysia), filename="GE13-WM")

# GE 12 (West Malaysia)

In [12]:
py.iplot(get_map("12", area=west_malaysia), filename="GE12-WM")

# GE 11 (West Malaysia)

In [7]:
py.iplot(get_map("11", area=west_malaysia), filename="GE11-WM")

# Other